# 🎯 MOUAADNET-ULTRA v2: Zero-Confidence FIX

**Complete rewrite to fix the all-blue heatmap issue**

### Key Fixes:
- ✅ **Decoupled Head**: Separate Heatmap/WH/Offset branches
- ✅ **Focal Loss**: Forces focus on person pixels
- ✅ **Gaussian Targets**: Soft blobs, not single pixels
- ✅ **Bias Init**: `-2.19` prevents zero output
- ✅ **1CycleLR**: 30% warmup, proper annealing

---

In [ ]:
#@title 1️⃣ Setup
!nvidia-smi
!pip install -q torch torchvision tqdm pycocotools opencv-python onnx

In [ ]:
#@title 2️⃣ Config
import os

WORK = '/teamspace/studios/this_studio' if os.path.exists('/teamspace') else '/content'
REPO = f'{WORK}/MouaadNet-Ultra'
DATA = f'{WORK}/coco'
CKPT = f'{WORK}/checkpoints_v2'

os.makedirs(DATA, exist_ok=True)
os.makedirs(CKPT, exist_ok=True)
print(f'Work: {WORK}')

In [ ]:
#@title 3️⃣ Clone/Update Repo
if not os.path.exists(REPO):
    !git clone https://github.com/mouuuuaad/MouaadNet-Ultra.git {REPO}
else:
    !cd {REPO} && git pull
os.chdir(REPO)
print(f'✅ {os.getcwd()}')

In [ ]:
#@title 4️⃣ Verify COCO Dataset
import shutil

def count_images(path):
    if not os.path.exists(path): return 0
    return len([f for f in os.listdir(path) if f.endswith('.jpg')])

EXPECTED_TRAIN = 118287
train_count = count_images(f'{DATA}/train2017')
val_count = count_images(f'{DATA}/val2017')

print(f'Train: {train_count}/{EXPECTED_TRAIN} ({100*train_count/EXPECTED_TRAIN:.1f}%)')
print(f'Val: {val_count}/5000')

if train_count < EXPECTED_TRAIN * 0.9:
    print('\n⚠️ COCO incomplete! Re-downloading...')
    shutil.rmtree(f'{DATA}/train2017', ignore_errors=True)
    !wget -c http://images.cocodataset.org/zips/train2017.zip -O {DATA}/train.zip
    !cd {DATA} && unzip -q train.zip && rm train.zip

if val_count < 4500:
    shutil.rmtree(f'{DATA}/val2017', ignore_errors=True)
    !wget -c http://images.cocodataset.org/zips/val2017.zip -O {DATA}/val.zip
    !cd {DATA} && unzip -q val.zip && rm val.zip

if not os.path.exists(f'{DATA}/annotations/instances_train2017.json'):
    !wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip -O {DATA}/ann.zip
    !cd {DATA} && unzip -q ann.zip && rm ann.zip

print('✅ COCO Ready!')

In [ ]:
#@title 5️⃣ Train V2 (FIXED VERSION)
EPOCHS = 50  #@param {type:"integer"}
BATCH = 32   #@param {type:"integer"}

!python training/train_detection_v2.py \
    --data {DATA} \
    --epochs {EPOCHS} \
    --batch-size {BATCH} \
    --save-dir {CKPT} \
    --export

In [ ]:
#@title 6️⃣ Visualize Heatmaps (Should be RED-HOT now!)
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, REPO)

# Load model
from training.train_detection_v2 import MouaadNetUltraV2, Detector

model = MouaadNetUltraV2()
ckpt = torch.load(f'{CKPT}/best.pt', map_location='cpu')
model.load_state_dict(ckpt['model_state_dict'])
model.eval()

# Test on a sample
img_path = f'{DATA}/val2017/000000001268.jpg'
img = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Run inference
detector = Detector(model, device='cpu')
tensor, scale, top, left = detector.preprocess(img_rgb)

with torch.no_grad():
    outputs = model(tensor)
    hm = torch.sigmoid(outputs['heatmap'][0, 0]).numpy()

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
ax1.imshow(cv2.resize(img_rgb, (256, 256)))
ax1.set_title('Input')
ax1.axis('off')

im = ax2.imshow(hm, cmap='hot', vmin=0, vmax=1)
ax2.set_title(f'Heatmap (max={hm.max():.4f})')
ax2.axis('off')
plt.colorbar(im, ax=ax2)

plt.tight_layout()
plt.savefig(f'{CKPT}/heatmap_test.png')
plt.show()

print(f'\nHeatmap max: {hm.max():.4f}')
print('Should be > 0.1 for successful training!')

In [ ]:
#@title 7️⃣ Run Detection
detections = detector.detect(img_rgb, threshold=0.3)

# Draw
vis = img_rgb.copy()
for det in detections:
    x1, y1, x2, y2 = [int(c) for c in det['box']]
    cv2.rectangle(vis, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(vis, f"{det['score']:.2f}", (x1, y1-5), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

plt.figure(figsize=(10, 8))
plt.imshow(vis)
plt.title(f'Detections: {len(detections)} persons')
plt.axis('off')
plt.savefig(f'{CKPT}/detection_test.png')
plt.show()

In [ ]:
#@title 8️⃣ Download Results
print('📦 Files:')
for f in os.listdir(CKPT):
    if f.endswith(('.pt', '.onnx', '.png')):
        print(f'   {f} ({os.path.getsize(f"{CKPT}/{f}")/1e6:.1f}MB)')

try:
    from google.colab import files
    files.download(f'{CKPT}/best.pt')
except:
    print(f'\nFiles at: {CKPT}')